# Managing the Conversation History
If converation history is left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. So it's important to add a step that limit the size of the messages you are passing in.



In [1]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)


In [2]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert AI assistant. Answer all question to the best of your abilities in {languages}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

chain = prompt|model

In [4]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
with_message_history = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")
config = {"configurable":{"session_id":"chat3"}}
# response = with_message_history.invoke(
#     {"messages":[HumanMessage(content="Hi my name is Aneesh")],"language":"Hindi"},
#     config=config
# )

'trim_messages': helps us reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial message

In [7]:
from langchain_core.messages import SystemMessage,trim_messages,AIMessage,HumanMessage

trimmer = trim_messages(
    max_tokens=70,
    strategy = "last",
    token_counter = model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="You are a good AI assistant"),
    HumanMessage(content="Hi! I am Aneesh"),
    AIMessage(content="Hi Aneesh!"),
    HumanMessage(content="I like vanilla ice cream."),
    AIMessage(content="Cool i like chocolate."),
    HumanMessage(content="what is 2+2"),
    AIMessage(content="the answer is 4"),
    HumanMessage(content="Thanks!"),
    AIMessage(content="No problem"),
    HumanMessage(content="Having fun"),
    AIMessage(content="Yes daddy. Ah yeah 🥵")
]
trimmer.invoke(messages)

[SystemMessage(content='You are a good AI assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Cool i like chocolate.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='the answer is 4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes daddy. Ah yeah 🥵', additional_kwargs={}, response_metadata={})]

In [13]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain = (
    RunnablePassthrough.assign(messages=itemgetter('messages')|trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages":messages+[HumanMessage(content="What did you say after i said having fun?")],
        "languages":"English"
    }
)
response.content

'I said "Yes daddy. Ah yeah 🥵" \n\nI apologize for that response. It was inappropriate and not aligned with my purpose as a helpful and harmless AI assistant. \n\nCan I try to make it up to you? Perhaps you\'d like to ask me another question or discuss a different topic?\n'